### Exercício: Análise Manual do Dataset Disney+

**Objetivo:**

Este exercício tem como objetivo desenvolver suas habilidades em manipulação manual de dados em Python, processando um dataset de shows da Disney+ sem o auxílio de bibliotecas de parsing como `csv` ou `pandas`. Você deverá extrair informações relevantes e gerar um relatório detalhado.

**Dataset:**

O dataset a ser utilizado está disponível no seguinte link:
[https://www.kaggle.com/datasets/eshummalik/disney](https://www.kaggle.com/datasets/eshummalik/disney)

**Instruções:**

1.  **Download do Dataset:** Baixe o arquivo do dataset manualmente a partir do link fornecido.
2.  **Carregamento Manual dos Dados:**
    *   Implemente uma função chamada `abrir_filmes_disney()` que será responsável por abrir o arquivo do dataset.
    *   Você **não** deve utilizar bibliotecas como `csv` ou `pandas` para ler o arquivo. A leitura deve ser feita linha por linha, utilizando as funcionalidades básicas de manipulação de arquivos em Python.
    *   Ignore linhas vazias encontradas no arquivo.
    *   Para cada linha do dataset, identifique se ela representa um filme (`type` coluna) ou uma série.
    *   Crie duas classes em Python: `Filme` e `Serie`. Cada linha do dataset deve ser mapeada para uma instância da classe `Filme` ou `Serie` correspondente, contendo seus respectivos atributos (baseados nas colunas do dataset). Implemente getters e setters para os atributos conforme necessário.
    *   Os campos que contêm o valor "N/A" no dataset devem ser tratados e armazenados como `None` nos objetos `Filme` ou `Serie`.
    *   A função `abrir_filmes_disney()` deve retornar uma lista contendo todos os objetos `Filme` e `Serie` criados a partir do dataset.
    *   Certifique-se de que o arquivo do dataset seja devidamente fechado ao final da execução da função `abrir_filmes_disney()`.
3.  **Geração do Relatório:**
    *   Implemente uma função chamada `gerar_relatorio()` que receberá como entrada a lista de objetos `Filme` e `Serie` retornada pela função `abrir_filmes_disney()`.
    *   Com base nos dados contidos nos objetos, calcule e inclua no relatório as seguintes informações:
        *   A média da nota de todos os filmes no IMDB.
        *   A média da nota de todos os filmes no Metascore.
        *   A média do número de votos de todos os filmes no IMDB.
        *   As 3 línguas mais usadas e as 3 línguas menos usadas no dataset.
        *   Os 3 atores que mais aparecem e os 3 atores que menos aparecem no dataset.
        *   O diretor com mais filmes no dataset.
        *   O diretor com o filme mais popular no IMDB (considerando a maior nota IMDB).
        *   O diretor com o filme mais popular no Metascore (considerando a maior nota Metascore).
        *   O ano em que mais filmes foram lançados.
        *   A pior série segundo a nota IMDB e a pior série segundo a nota Metascore.
        *   Uma lista de filmes que possuem mais de um lançamento (considerados "remakes" ou diferentes versões no dataset, identificados por títulos iguais mas anos de lançamento diferentes).
    *   O relatório de saída deve ser salvo em um arquivo texto chamado `relatorio-disney.txt` no mesmo diretório do script.
4.  **Estrutura do Código:** Organize seu código de forma modular, utilizando as funções `abrir_filmes_disney()` e `gerar_relatorio()` conforme especificado. Você não é obrigado a passar parâmetros para essas funções, mas pode fazê-lo se julgar necessário para uma melhor organização do código.

In [12]:
class Filme:
    def __init__(self, dados):
        # SÓ OS CAMPOS QUE VAMOS USAR NO RELATÓRIO
        self.title = dados.get('title')
        self.type = dados.get('type')
        self.year = dados.get('year')
        self.imdb_rating = dados.get('imdb_rating')
        self.metascore = dados.get('metascore')
        self.imdb_votes = dados.get('imdb_votes')
        self.language = dados.get('language')
        self.actors = dados.get('actors')
        self.director = dados.get('director')

class Serie:
    def __init__(self, dados):
        # MESMOS campos essenciais
        self.title = dados.get('title')
        self.type = dados.get('type')
        self.year = dados.get('year')
        self.imdb_rating = dados.get('imdb_rating')
        self.metascore = dados.get('metascore')
        self.imdb_votes = dados.get('imdb_votes')
        self.language = dados.get('language')
        self.actors = dados.get('actors')
        self.director = dados.get('director')

def encontrar_remakes(filmes):
    titulos = {}
    
    for filme in filmes:
        if filme.title:
            if filme.title in titulos:
                titulos[filme.title].append(filme)
            else:
                titulos[filme.title] = [filme]
    
    # Filtrar só títulos com mais de um filme
    remakes = {titulo: filmes for titulo, filmes in titulos.items() if len(filmes) > 1}
    return remakes

def piores_series(series):
    # Filtrar séries que têm nota válida (não None e não string vazia)
    series_com_imdb = [s for s in series if s.imdb_rating and s.imdb_rating != 'N/A']
    series_com_metascore = [s for s in series if s.metascore and s.metascore != 'N/A']
    
    pior_imdb = None
    pior_metascore = None
    
    if series_com_imdb:
        pior_imdb = min(series_com_imdb, key=lambda x: float(x.imdb_rating))
    
    if series_com_metascore:
        pior_metascore = min(series_com_metascore, key=lambda x: float(x.metascore))
    return pior_imdb, pior_metascore

def ano_mais_lancamentos(dados):
    contagem_anos = {}
    
    for item in dados:
        if item.year:
            # Pegar só o ano (pode ter formato "2018–")
            ano = item.year.split('–')[0].strip()
            if ano.isdigit():
                contagem_anos[ano] = contagem_anos.get(ano, 0) + 1
    
    if contagem_anos:
        return max(contagem_anos.items(), key=lambda x: x[1])
    return None

def diretor_filme_mais_popular(filmes, tipo_nota='imdb'):
    """
    tipo_nota: 'imdb' ou 'metascore'
    """
    filme_maior_nota = None
    maior_nota = 0
    
    for filme in filmes:
        if tipo_nota == 'imdb':
            nota_str = filme.imdb_rating
        else:  # metascore
            nota_str = filme.metascore
        
        # Verificar se a nota existe e é válida
        if nota_str and nota_str != 'N/A':
            try:
                nota = float(nota_str)
                if nota > maior_nota:
                    maior_nota = nota
                    filme_maior_nota = filme
            except ValueError:
                continue
    
    if filme_maior_nota and filme_maior_nota.director:
        diretor = filme_maior_nota.director.split(',')[0].strip()
        diretor = diretor.replace('"', '')  # Remover aspas
        return diretor, maior_nota
    return None, 0

def diretor_mais_filmes(dados):
    contagem_diretores = {}
    
    for item in dados:
        if item.director and isinstance(item, Filme):
            diretores = [d.strip() for d in item.director.split(',')]
            for diretor in diretores:
                contagem_diretores[diretor] = contagem_diretores.get(diretor, 0) + 1
    
    if contagem_diretores:
        return max(contagem_diretores.items(), key=lambda x: x[1])
    return None

def atores_mais_menos_frequentes(dados):
    contagem_atores = {}
    
    for item in dados:
        if item.actors:
            atores = [ator.strip() for ator in item.actors.split(',')]
            for ator in atores:
                contagem_atores[ator] = contagem_atores.get(ator, 0) + 1
    
    atores_ordenados = sorted(contagem_atores.items(), key=lambda x: x[1], reverse=True)
    
    mais_frequentes = atores_ordenados[:3]
    menos_frequentes = atores_ordenados[-3:] if len(atores_ordenados) >= 3 else atores_ordenados
    
    return mais_frequentes, menos_frequentes

def linguas_mais_menos_usadas(dados):
    contagem_linguas = {}
    
    for item in dados:
        if item.language:  # Ignorar None
            # Remover aspas extras e normalizar
            linguas_str = item.language.replace('"', '').strip()
            linguas = [lingua.strip() for lingua in linguas_str.split(',')]
            
            for lingua in linguas:
                # Normalizar: primeira letra maiúscula, resto minúscula
                lingua_normalizada = lingua.capitalize()
                contagem_linguas[lingua_normalizada] = contagem_linguas.get(lingua_normalizada, 0) + 1
    
    # Ordenar por frequência (maior para menor)
    linguas_ordenadas = sorted(contagem_linguas.items(), key=lambda x: x[1], reverse=True)
    
    # Pegar 3 mais e 3 menos usadas
    mais_usadas = linguas_ordenadas[:3]  # Primeiros 3
    menos_usadas = linguas_ordenadas[-3:] if len(linguas_ordenadas) >= 3 else linguas_ordenadas  # Últimos 3
    return mais_usadas, menos_usadas

def media_votos(filmes):
    n_votos=[]
    for filme in filmes:
        if filme.imdb_votes: #Ignorar None
            votos_str=filme.imdb_votes.replace(',','').replace('"',"")
            votos=int(votos_str)
            n_votos.append(votos)
    
    med_voto=sum(n_votos)/len(n_votos)
    return med_voto

def media_filmes(filmes):
    # Média dos filmes pelo IMDB e MetaScore
    notas_imdb=[]
    notas_metascore=[]
    for filme in filmes:
        if filme.imdb_rating: #Ignorar None
            nota=float(filme.imdb_rating)
            notas_imdb.append(nota)
        if filme.metascore: #Ignorar None
            nota=float(filme.metascore)
            notas_metascore.append(nota)

    media_imdb = sum(notas_imdb)/len(notas_imdb)
    media_metascore = sum(notas_metascore)/len(notas_metascore)
    return media_imdb,media_metascore

def dividir_colunas(linha):
# Função para dividir as colunas e coletar os valores separados por vígula quando não estão entre aspas
    valores = []
    i = 0
    dentro_de_aspas = False
    valor_atual = ""
    
    while i < len(linha):
        char = linha[i]
        
        if char == '"':
            dentro_de_aspas = not dentro_de_aspas
            valor_atual += char
        elif char == ',' and not dentro_de_aspas:
            valores.append(valor_atual)
            valor_atual = ""
        else:
            valor_atual += char
        
        i += 1
    
    if valor_atual:
        valores.append(valor_atual)
    
    return valores

def abrir_filmes_disney():
    # Abrir o arquivo e deixá-lo consultável
    conteudo = []
    
    with open('disney_plus_shows.csv', 'r', encoding='utf-8') as f:
        # 1. Ler cabeçalho
        header = f.readline().strip()
        colunas = dividir_colunas(header)
        
        # 2. Ler cada linha
        for linha in f:
            linha = linha.strip()
            if linha == "":
                continue
            
            # 3. Dividir valores
            valores = dividir_colunas(linha)
            
            # 4. Criar dicionário
            registro = {}
            for i, coluna in enumerate(colunas):
                if i < len(valores):
                    valor = valores[i]
                    if valor == 'N/A' or valor == '':
                        registro[coluna] = None
                    else:
                        registro[coluna] = valor
                else:
                    registro[coluna] = None
            
            # 5. Registrando nas Class de Filme ou Serie
            if registro.get('type') == 'movie':
                conteudo.append(Filme(registro))
            else:
                conteudo.append(Serie(registro))
    return conteudo

def gerar_relatorio(dados):
    # 1. Separar filmes e séries
    filmes = [f for f in dados if isinstance(f, Filme)]
    series = [s for s in dados if isinstance(s, Serie)]
    
    # 2. Abrir arquivo para escrever
    with open('relatorio-disney.txt', 'w', encoding='utf-8') as relatorio:
        
        # 3. Escrever cada seção do relatório
        relatorio.write('=== RELATÓRIO DISNEY+ ===\n\n')
        relatorio.write('1. Médias pelos Sites \n')
        # Item 1
        media_imdb, media_metascore = media_filmes(filmes)
        relatorio.write(f'A média dos filmes pelo IMDB é: {media_imdb:.2f} \n')
        relatorio.write(f'A média dos filmes pelo MetaScore é: {media_metascore:.2f} \n')

        # Item 2
        relatorio.write('\n 2. Médias do número de votos no IMDB \n')
        med_voto=media_votos(filmes)
        relatorio.write(f'Média de votos: {int(med_voto)} \n')

        # Item 3
        relatorio.write("\n3. LÍNGUAS MAIS E MENOS USADAS:\n")
        mais_usadas, menos_usadas = linguas_mais_menos_usadas(dados)
        relatorio.write("   Mais usadas:\n")
        for lingua, count in mais_usadas:
            relatorio.write(f"     - {lingua}: {count} vezes\n")

        relatorio.write("   Menos usadas:\n")
        for lingua, count in menos_usadas:
            relatorio.write(f"     - {lingua}: {count} vezes\n")

        # Item 4
        relatorio.write("\n4. ATORES MAIS E MENOS FREQUENTES:\n")
        mais_atores, menos_atores = atores_mais_menos_frequentes(dados)
        relatorio.write("   Mais frequentes:\n")
        for ator, count in mais_atores:
            relatorio.write(f"     - {ator}: {count} filmes/séries\n")
        relatorio.write("   Menos frequentes:\n")
        for ator, count in menos_atores:
            relatorio.write(f"     - {ator}: {count} filmes/séries\n")

        # Item 5
        relatorio.write("\n5. DIRETOR COM MAIS FILMES:\n")
        diretor_mais = diretor_mais_filmes(dados)
        if diretor_mais:
            relatorio.write(f"   {diretor_mais[0]}: {diretor_mais[1]} filmes\n")
        else:
            relatorio.write("   Nenhum diretor encontrado\n")

        # Item 6 - Diretor com filme mais popular (IMDB)
        relatorio.write("\n6. DIRETOR COM FILME MAIS POPULAR NO IMDB:\n")
        diretor_imdb, nota_imdb = diretor_filme_mais_popular(filmes, 'imdb')
        if diretor_imdb:
            relatorio.write(f"   {diretor_imdb}: nota {nota_imdb:.1f}\n")
        else:
            relatorio.write("   Nenhum filme com nota IMDB encontrado\n")

        # Item 7 - Diretor com filme mais popular (Metascore)
        relatorio.write("\n7. DIRETOR COM FILME MAIS POPULAR NO METASCORE:\n")
        diretor_meta, nota_meta = diretor_filme_mais_popular(filmes, 'metascore')
        if diretor_meta:
            relatorio.write(f"   {diretor_meta}: nota {nota_meta:.0f}\n")
        else:
            relatorio.write("   Nenhum filme com Metascore encontrado\n")

        # Item 8 
        relatorio.write("\n8. ANO COM MAIS LANÇAMENTOS:\n")
        ano_mais = ano_mais_lancamentos(dados)
        if ano_mais:
            relatorio.write(f"   {ano_mais[0]}: {ano_mais[1]} lançamentos\n")
        else:
            relatorio.write("   Nenhum ano encontrado\n")

        # Item 9
        relatorio.write("\n9. PIORES SÉRIES:\n")
        pior_serie_imdb, pior_serie_metascore = piores_series(series)
        if pior_serie_imdb:
            relatorio.write(f"   Pior série no IMDB: {pior_serie_imdb.title} - nota {pior_serie_imdb.imdb_rating}\n")
        else:
            relatorio.write("   Nenhuma série com nota IMDB encontrada\n")
        
        if pior_serie_metascore:
            relatorio.write(f"   Pior série no Metascore: {pior_serie_metascore.title} - nota {pior_serie_metascore.metascore}\n")
        else:
            relatorio.write("   Nenhuma série com Metascore encontrada\n")

        # Item 10
        relatorio.write("\n10. FILMES COM MÚLTIPLOS LANÇAMENTOS (REMAKES):\n")
        remakes = encontrar_remakes(filmes)
        if remakes:
            for titulo, filmes_list in remakes.items():
                anos = [f.year for f in filmes_list if f.year]
                relatorio.write(f"   {titulo}: {', '.join(anos)}\n")
        else:
            relatorio.write("   Nenhum remake encontrado\n")
        
       
dados=abrir_filmes_disney()
gerar_relatorio(dados)



    